# Zbuduj model klasyfikacji: Pyszne azjatyckie i indyjskie kuchnie


## Klasyfikatory kuchni 2

W tej drugiej lekcji dotyczącej klasyfikacji, zbadamy `więcej sposobów` klasyfikowania danych kategorycznych. Dowiemy się również, jakie konsekwencje niesie za sobą wybór jednego klasyfikatora zamiast innego.

### [**Quiz przed wykładem**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Wymagania wstępne**

Zakładamy, że ukończyłeś poprzednie lekcje, ponieważ będziemy kontynuować niektóre wcześniej omówione pojęcia.

Do tej lekcji będziemy potrzebować następujących pakietów:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) to [zbiór pakietów R](https://www.tidyverse.org/packages) zaprojektowany, aby uczynić analizę danych szybszą, łatwiejszą i bardziej przyjemną!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) to [framework](https://www.tidymodels.org/packages/) składający się z pakietów do modelowania i uczenia maszynowego.

-   `themis`: [pakiet themis](https://themis.tidymodels.org/) dostarcza dodatkowe kroki w przepisach do radzenia sobie z niezrównoważonymi danymi.

Możesz je zainstalować za pomocą:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternatywnie, poniższy skrypt sprawdza, czy masz wymagane pakiety do ukończenia tego modułu i instaluje je, jeśli ich brakuje.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Mapa klasyfikacji**

W naszej [poprzedniej lekcji](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) próbowaliśmy odpowiedzieć na pytanie: jak wybrać pomiędzy wieloma modelami? W dużej mierze zależy to od charakterystyki danych oraz rodzaju problemu, który chcemy rozwiązać (na przykład klasyfikacja czy regresja?).

Wcześniej dowiedzieliśmy się o różnych opcjach klasyfikacji danych, korzystając z arkusza pomocy Microsoftu. Framework Machine Learning w Pythonie, Scikit-learn, oferuje podobny, ale bardziej szczegółowy arkusz pomocy, który może dodatkowo pomóc w zawężeniu wyboru estymatorów (inaczej klasyfikatorów):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Wskazówka: [odwiedź tę mapę online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) i klikaj po ścieżkach, aby przeczytać dokumentację.
>
> [Strona referencyjna Tidymodels](https://www.tidymodels.org/find/parsnip/#models) również oferuje doskonałą dokumentację dotyczącą różnych typów modeli.

### **Plan** 🗺️

Ta mapa jest bardzo pomocna, gdy dobrze rozumiesz swoje dane, ponieważ możesz „przejść” jej ścieżkami, aby podjąć decyzję:

-   Mamy \>50 próbek

-   Chcemy przewidzieć kategorię

-   Mamy dane z etykietami

-   Mamy mniej niż 100 tys. próbek

-   ✨ Możemy wybrać Linear SVC

-   Jeśli to się nie sprawdzi, ponieważ mamy dane numeryczne

    -   Możemy spróbować ✨ KNeighbors Classifier

        -   Jeśli to się nie sprawdzi, spróbuj ✨ SVC i ✨ Ensemble Classifiers

To bardzo pomocna ścieżka do podążania. Teraz przejdźmy do działania, korzystając z frameworka modelowania [tidymodels](https://www.tidymodels.org/): spójnej i elastycznej kolekcji pakietów R, opracowanej w celu promowania dobrych praktyk statystycznych 😊.

## 2. Podziel dane i zajmij się niezrównoważonym zestawem danych.

Z naszych poprzednich lekcji dowiedzieliśmy się, że istnieje zestaw wspólnych składników w różnych kuchniach. Ponadto zauważyliśmy nierównomierny rozkład liczby kuchni.

Poradzimy sobie z tym, wykonując następujące kroki:

-   Usuniemy najbardziej popularne składniki, które powodują zamieszanie między różnymi kuchniami, używając `dplyr::select()`.

-   Użyjemy `recipe`, który wstępnie przetwarza dane, aby przygotować je do modelowania, stosując algorytm `over-sampling`.

Omówiliśmy powyższe w poprzedniej lekcji, więc to powinno być proste 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Radzenie sobie z niezrównoważonymi danymi

Niezrównoważone dane często negatywnie wpływają na wydajność modelu. Wiele modeli działa najlepiej, gdy liczba obserwacji jest równa, przez co mają trudności z przetwarzaniem niezrównoważonych danych.

Istnieją dwa główne sposoby radzenia sobie z niezrównoważonymi zbiorami danych:

-   dodawanie obserwacji do klasy mniejszościowej: `Over-sampling`, np. za pomocą algorytmu SMOTE, który syntetycznie generuje nowe przykłady klasy mniejszościowej, wykorzystując najbliższych sąsiadów tych przypadków.

-   usuwanie obserwacji z klasy większościowej: `Under-sampling`

W naszej poprzedniej lekcji pokazaliśmy, jak radzić sobie z niezrównoważonymi zbiorami danych, korzystając z `recipe`. Recipe można traktować jako plan działania, który opisuje, jakie kroki należy zastosować do zbioru danych, aby przygotować go do analizy. W naszym przypadku chcemy uzyskać równomierny rozkład liczby naszych kuchni w `training set`. Przejdźmy do działania.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Teraz jesteśmy gotowi do trenowania modeli 👩‍💻👨‍💻!

## 3. Poza modelami regresji wielomianowej

W naszej poprzedniej lekcji przyjrzeliśmy się modelom regresji wielomianowej. Zbadajmy teraz bardziej elastyczne modele klasyfikacji.

### Maszyny wektorów nośnych

W kontekście klasyfikacji, `Maszyny wektorów nośnych` to technika uczenia maszynowego, która stara się znaleźć *hiperpłaszczyznę*, która "najlepiej" oddziela klasy. Spójrzmy na prosty przykład:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ nie oddziela klas. H2~ oddziela je, ale tylko z małym marginesem. H3~ oddziela je z maksymalnym marginesem.

#### Liniowy Klasyfikator Wektorów Nośnych

Klasyfikacja za pomocą wektorów nośnych (SVC) jest częścią rodziny metod uczenia maszynowego opartych na wektorach nośnych. W SVC hiperpłaszczyzna jest wybierana tak, aby poprawnie oddzielić `większość` obserwacji treningowych, ale `może błędnie sklasyfikować` kilka obserwacji. Pozwalając niektórym punktom znaleźć się po niewłaściwej stronie, SVM staje się bardziej odporny na wartości odstające, co prowadzi do lepszej uogólnienia na nowe dane. Parametr regulujący to naruszenie nazywany jest `kosztem` i ma domyślną wartość 1 (zobacz `help("svm_poly")`).

Stwórzmy liniowy SVC, ustawiając `degree = 1` w modelu SVM z wielomianem.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Teraz, gdy uchwyciliśmy kroki wstępnego przetwarzania i specyfikację modelu w *workflow*, możemy przejść do trenowania liniowego SVC i jednocześnie ocenić wyniki. Aby uzyskać metryki wydajności, stwórzmy zestaw metryk, który oceni: `accuracy`, `sensitivity`, `Positive Predicted Value` oraz `F Measure`.

> `augment()` doda kolumny z przewidywaniami do podanych danych.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Maszyna wektorów nośnych

Maszyna wektorów nośnych (SVM) jest rozszerzeniem klasyfikatora wektorów nośnych, które pozwala na uwzględnienie nieliniowej granicy między klasami. W istocie, SVM wykorzystują *sztuczkę z jądrem*, aby powiększyć przestrzeń cech i dostosować się do nieliniowych zależności między klasami. Jedną z popularnych i niezwykle elastycznych funkcji jądra stosowanych w SVM jest *funkcja bazowa radialna*. Zobaczmy, jak poradzi sobie z naszymi danymi.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Dużo lepiej 🤩!

> ✅ Proszę zobaczyć:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> dla dalszej lektury.

### Klasyfikatory najbliższego sąsiada

Algorytm *K*-najbliższych sąsiadów (KNN) przewiduje każdą obserwację na podstawie jej *podobieństwa* do innych obserwacji.

Dopasujmy go do naszych danych.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Wydaje się, że ten model nie działa zbyt dobrze. Prawdopodobnie zmiana argumentów modelu (zobacz `help("nearest_neighbor")`) poprawi jego wydajność. Koniecznie wypróbuj tę opcję.

> ✅ Zobacz:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> aby dowiedzieć się więcej o klasyfikatorach *K*-Nearest Neighbors.

### Klasyfikatory zespołowe

Algorytmy zespołowe działają poprzez łączenie wielu bazowych estymatorów w celu stworzenia optymalnego modelu, wykorzystując jedną z dwóch metod:

`bagging`: zastosowanie *funkcji uśredniającej* do zbioru modeli bazowych

`boosting`: budowanie sekwencji modeli, które wzajemnie się uzupełniają, aby poprawić wydajność predykcyjną.

Zacznijmy od wypróbowania modelu Random Forest, który tworzy dużą liczbę drzew decyzyjnych, a następnie stosuje funkcję uśredniającą, aby uzyskać lepszy model ogólny.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Dobra robota 👏!

Spróbujmy również modelu Boosted Tree.

Boosted Tree to metoda zespołowa, która tworzy serię sekwencyjnych drzew decyzyjnych, gdzie każde drzewo zależy od wyników poprzednich drzew, aby stopniowo zmniejszać błąd. Skupia się na wagach błędnie sklasyfikowanych elementów i dostosowuje dopasowanie dla kolejnego klasyfikatora, aby je poprawić.

Istnieją różne sposoby dopasowania tego modelu (zobacz `help("boost_tree")`). W tym przykładzie dopasujemy Boosted trees za pomocą silnika `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Proszę zobaczyć:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Omawia model AdaBoost, który jest dobrą alternatywą dla xgboost.
>
> aby dowiedzieć się więcej o klasyfikatorach zespołowych.

## 4. Dodatkowo - porównanie wielu modeli

W tym laboratorium dopasowaliśmy całkiem sporo modeli 🙌. Może to stać się żmudne lub uciążliwe, jeśli musimy tworzyć wiele przepływów pracy z różnych zestawów procesorów wstępnych i/lub specyfikacji modeli, a następnie obliczać metryki wydajności jeden po drugim.

Zobaczmy, czy możemy rozwiązać ten problem, tworząc funkcję, która dopasowuje listę przepływów pracy do zestawu treningowego, a następnie zwraca metryki wydajności na podstawie zestawu testowego. Skorzystamy z `map()` i `map_dfr()` z pakietu [purrr](https://purrr.tidyverse.org/), aby zastosować funkcje do każdego elementu w liście.

> Funkcje [`map()`](https://purrr.tidyverse.org/reference/map.html) pozwalają zastąpić wiele pętli for kodem, który jest zarówno bardziej zwięzły, jak i łatwiejszy do odczytania. Najlepszym miejscem do nauki o funkcjach [`map()`](https://purrr.tidyverse.org/reference/map.html) jest [rozdział o iteracji](http://r4ds.had.co.nz/iteration.html) w R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) umożliwia użytkownikom tworzenie i łatwe dopasowywanie dużej liczby modeli, ale jest głównie zaprojektowany do pracy z technikami resamplingu, takimi jak `cross-validation`, podejście, które dopiero omówimy.

## **🚀Wyzwanie**

Każda z tych technik ma wiele parametrów, które można dostosować, na przykład `cost` w SVM, `neighbors` w KNN, `mtry` (losowo wybrane predyktory) w Random Forest.

Zbadaj domyślne wartości parametrów dla każdej z nich i zastanów się, co zmiana tych parametrów oznaczałaby dla jakości modelu.

Aby dowiedzieć się więcej o konkretnym modelu i jego parametrach, użyj: `help("model")`, np. `help("rand_forest")`.

> W praktyce zazwyczaj *szacujemy* *najlepsze wartości* tych parametrów, trenując wiele modeli na `symulowanym zestawie danych` i mierząc, jak dobrze te modele się sprawdzają. Ten proces nazywa się **strojenie**.

### [**Quiz po wykładzie**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Przegląd i samodzielna nauka**

W tych lekcjach pojawia się wiele specjalistycznych terminów, więc poświęć chwilę na przejrzenie [tej listy](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) przydatnej terminologii!

#### PODZIĘKOWANIA DLA:

[`Allison Horst`](https://twitter.com/allison_horst/) za stworzenie niesamowitych ilustracji, które sprawiają, że R jest bardziej przyjazny i angażujący. Więcej ilustracji znajdziesz w jej [galerii](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) i [Jen Looper](https://www.twitter.com/jenlooper) za stworzenie oryginalnej wersji tego modułu w Pythonie ♥️

Miłej nauki,

[Eric](https://twitter.com/ericntay), Złoty Ambasador Studentów Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Ilustracja autorstwa @allison_horst</figcaption>



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
